In [4]:
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set seed value for reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Data directories
train_dir = 'train/'
val_dir = 'val/'
test_dir = 'test/'

# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

# Model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

# Model training
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples//train_data.batch_size,
    epochs=10,
    validation_data=val_data,
    validation_steps=val_data.samples//val_data.batch_size)

# Model evaluation
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_loss, test_acc = model.evaluate(test_data, steps=test_data.samples//test_data.batch_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# Making predictions on new images
new_image_path = '/content/passport size photo nithin_page-0001.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis=0)
new_image /= 255.

prediction = model.predict(new_image)
if prediction > 0.5:
    print('Fake Image')
else:
    print('Real Image')


FileNotFoundError: ignored